# Question 1.1 Data Transformation

In [9]:
import pandas as pd
import numpy as np
import os
import tarfile
import glob
from datetime import datetime
import math
import csv

### Getting all unique parameters

In [10]:
# Define paths
csv_directory = "data/set-a/"
unique_parameters = set()

# Looping through each patient file and gathering the parameter name
for filename in os.listdir(csv_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(csv_directory, filename)
        
        with open(file_path, mode='r') as file:
            reader = csv.reader(file)
            for row in reader:
                # Get row parameter name and add to set
                parameter = row[1]
                unique_parameters.add(parameter)

unique_parameters = list(unique_parameters)
unique_parameters.sort()
unique_parameters.remove("ICUType")

print("Unique Parameters:", unique_parameters)
print(len(unique_parameters))

Unique Parameters: ['ALP', 'ALT', 'AST', 'Age', 'Albumin', 'BUN', 'Bilirubin', 'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'HR', 'Height', 'K', 'Lactate', 'MAP', 'MechVent', 'Mg', 'NIDiasABP', 'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Parameter', 'Platelets', 'RecordID', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC', 'Weight', 'pH']
42


### Creating a large table for all patients

In [19]:
# Run this code for each of the sets once to create corresponding .parquet file (change folder names in lines 2 and 40 to "set-x")
file_paths = glob.glob("data/set-c/*.txt")

processed_dfs = []

# Loops through each of the patient .txt file and creates corresponding table. In the end, appends all tables to one big table and saves it as .parquet file
for file in file_paths:
    df = pd.read_csv(file)
    
    # For rounding to the next hour
    df[['Hour', 'Minute']] = df['Time'].str.split(':', expand=True).astype(int)
    df['RoundedHour'] = np.where((df['Hour'] == 0) & (df['Minute'] == 0), 0, ((df['Hour'] * 60 + df['Minute']) // 60 + 1) * 60)
    df['Time'] = df['RoundedHour'].apply(lambda x: f"{x//60:02}:00")

    # If we have two measurements during one hour (e.g, at 00:37 and at 00:57), we take the mean and get one value for the timestamp 01:00
    df_pivot = df.pivot_table(index='Time', columns='Parameter', values='Value', aggfunc='mean').reset_index()

    # Ensure all hours from 00:00 to 48:00 are present
    full_hours = pd.DataFrame({"Time": [f"{h:02}:00" for h in range(49)]})
    df_final = full_hours.merge(df_pivot, on="Time", how="left")

    # Add missing parameters from unique_parameters with NaN values (since not all parameters are measured on one patient)
    for param in unique_parameters:
        if param not in df_final.columns:
            df_final[param] = np.nan

    # Enforce static values to be persistent over all timestamps
    df_final["Age"] = df_final["Age"][0]
    df_final["Height"] = df_final["Height"][0]
    df_final["Weight"] = df_final["Weight"][0]
    df_final["Gender"] = df_final["Gender"][0]
    df_final["RecordID"] = df_final["RecordID"][0]

    processed_dfs.append(df_final)

final_df = pd.concat(processed_dfs, ignore_index=True)

# Saving result and displaying a preview
final_df.to_parquet('processed-data/processed-set-c.parquet', index=False)
final_df.head()

,Time,Age,BUN,Creatinine,DiasABP,FiO2,GCS,Gender,Glucose,HCO3,...,ALP,ALT,AST,Albumin,Bilirubin,Cholesterol,MechVent,Parameter,TroponinI,TroponinT
0,00:00,58.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01:00,58.0,NaN,NaN,53.000000,NaN,15.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02:00,58.0,11.0,0.5,56.666667,NaN,15.0,1.0,190.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,03:00,58.0,10.0,0.5,57.500000,NaN,15.0,1.0,160.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04:00,58.0,NaN,NaN,53.000000,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Getting the labels from Outcomes-x.txt files and creating corresponding .parquet files

In [22]:
# Run this code for each of the sets once to create corresponding .parquet file (change folder names in lines 2 and 4 to "set-x")
df = pd.read_csv('data/Outcomes-c.txt')
df_labels = df[["RecordID","In-hospital_death"]]
df_labels.to_parquet('processed-outcomes-c.parquet', index=False)

In [23]:
df_labels.head()

,RecordID,In-hospital_death
0,152871,0
1,152873,1
2,152875,0
3,152878,0
4,152882,0


# Q1.2 Exploratory Data Analysis